# Intro to Flask, Lesson 7

## APIs 

### REST APIs

So far, we only have visited sites manually, so we had put user information into templates and views. But we have yet to consider what to do if we wanted to just directly access information, or wanted a server to directly access our database.

![SegmentLocal](https://media3.giphy.com/media/d2Z0zWe67TDcbCzC/giphy.gif "segment")

This is where the software architecture style __REST__ comes in, with stands for:

__RE__ representational 

__S__ state

__T__ transfer 

This allows us to provide interactions between different computer systems online in a standardized way.

There are only certain times when you will want to use REST though, for example:

If you are making a website for something and it will mostly just be photos and concrete information, then you will likely not need REST here.

But if you are making a site with a lot of different information that changes often, such as an online shop, and users want to filter information, users could either manually scrape through the website with their own programming scripts, or you can get a REST API for them to use!

The easiest and most popular way to use this API is with the Flask-Restful Library. 

To use this need to understand some of the terminology for APIs, in a similar to way to that with Databases (and which you might have seen in your terminal when running the web page you made in lesson 6):

    To create with API: POST
    To read with API: GET
    To update with API: PUT
    To delete with API: DELETE

A helpful service to use with REST is Postman, which you will want to install to use the above operations. The website is called Postman, and there you just want to hit Download and select the operating systeming you are running on (Mac/OS or Linux).

So now we need to learn how actually learn how to send back information using REST, and the most common format for this is to use json, which as you will see is very similar to python dictionary

Before you begin to work on a flask application to try this, you need to make sure your environment is activated and that you: 

In [ ]:
pip install Flask-Restful

Then for a really simple example:

In [ ]:
# simpleapi.py
from flask import Flask 
from flask_restful import Resource, Api
# Resource allows you to create a resource to connect to, 
# and Api will allow that resource to actually connect!

app = Flask(__name__)

api = Api(app)
# Wrapping our application with an api call

class HelloWorld(Resource):
    # Created a class which inherits from Resource
    
    def get(self):
        return {'hello': 'world'}
    
api.add_resource(HelloWorld, '/')
# This adds a resources, and by passing the class and the homepage denoter, 
# it connects the two 

if __name__ == '__main__':
    app.run(debug = True)

source Jose Portilla

First run this, then open up Postman and create a new __collection__! A collection is basically a folder holding a bunch of requests. After creating this, you will see the name of the collection on the left hand side, with three dots that when you hover over, you get a variety of options in regards to the collection (share collection, modify permissions, etc).

![SegmentLocal](https://cdn.softwaretestinghelp.com/wp-content/qa/uploads/2019/10/Postman-addRequestToCollection.jpg "segment")

What you want to do here is choose to __add request__.

Then you can click on your new request, and in the search bar it provides for you, choose the GET option, and provide the URL given to you by your running app. This should return hello world!

Now if we want to build a more complicated example:

In [ ]:
# simpleapi.py
from flask import Flask
from flask_restful import Resource, Api

app = Flask(__name__)
api = Api(app)

puppies = []
# This is creating an empty list of puppies that will eventually be formated like:
# [{'name':'Rufus'}, {'name':'Atticus'},...]

class PuppyNames(Resource):
    
    def get(self, name):
        
        for pup in puppies:
            if pup['name'] == name:
                return pup
            # Goes through every puppy and finds the one with the given name 
        
        return {'name': None}
        # If no puppy has that name 
    
    def post(self, name):
        
        pup = {'name': name}
        # We will pass in a URL that ends in /name, where the name is what we are using
        
        puppies.append(pup)
        # Adds the pup to the end of the list of puppies
        
        return pup
    
    def delete(self, name):
        for index, pup in enumerate(puppies):
            # index is the index position and enumerate basically passes a counter
            
            if pup['name'] == name:
                deleted_pup = puppies.pop(ind)
                return {'note':'delete success'}
    # All the get, post, delete methods have to have the same parameters

class AllNames(Resource):
    
    def get(self):
        return {'puppies': puppies}
        # Returns the full list of puppies
    

api.add_resource(PuppyNames, '/puppy/<string:name>')
api.add_resource(AllNames, '/puppies')

if __name__ == '__main__':
    app.run(debug = True)
    

source Jose Portilla

So when you use Postman as we did before, you can use __POST__ to add puppies to the list. If you want to add a puppy whose name is Atticus, for example, under "method," select the __POST__ tab. Add the URL in the box, then type __/puppy/Atticus__, so you should be entering __URL/puppy/Atticus__.

You can retrieve a puppy's information by name by using the same URL (with the addition of __/puppy/puppy_name__) but changing the method to __GET__.

Then you can delete by once again using the same URL (with the addition of __/puppy/puppy_name__) but changing the method to __DELETE__.

If you want to display a list of all the puppies, you would just give the same URL and add __/puppies__.

### Flask JWT

The __Flask-JWT__ library, which stands for Jason Webb Token Library, can be used tp require authorizaation before a REST API is ever called 

In [ ]:
pip install Flask-JWT

If we use the example from above, we just need to add a few lines to the main file, and then two more files

So the two other files we need are:

In [ ]:
# user.py
class User():
    
    def __init__(self, id, username, password):
        
        self.id = id
        self.username = username
        self.password = password
        
    def __str__(self):
        return "User Id: {self.id}"

In [ ]:
# secure_check.py
from user import User
# Imports the User class we just created above

users = [User(1, "Kelly", "mypassword"), User(2, "Mimi", "secret")]
# This creates some users

username_table = {u.username: u for u in users}
# For every user, grab username and link it to that user to create the table and be able to look things up by username
userid_table = {u.id: u for u in users}

def authenticate(username, password):
    # Checks if user still exists and if so, returns the user
    
    user = username_table.get(username, None)
    # The None signifies that if the user does not exist, return None
    
    if user and password == user.password:
        # If the user is not None, therefore exists, and the psasword is correct
        return user

def identity(payload):
    user_id = payload['identity']
    return userid_table.get(user_id,None)

Source Jose Portilla

Then in the original file with PuppyClass and AllNames, we need to add two more imports:

In [ ]:
from secure_check import authenticate, identity
from flask_jwt import JWT, jwt_required

Then below where you create the Flask application:

In [ ]:
app.config['SECRET_KEY'] = 'secretkey'

jwt = JWT(app, authenticate, identity)

If you want to make something exclusively available to authorized users, add a decorator before each exclusive function.

For example, if you want all the names to be printed only for authenticated users, your AllNames class would be like this:

In [ ]:
class AllNames(Resource):
    
    @jwt_required()
    def get(self):
        return {'puppies': puppies}

## With Databases!

![SegmentLocal](https://media2.giphy.com/media/3q3QK6KyDVUBzih7hB/giphy.gif "segment")

To make this work with databases, we just have to change a few things about our previous code!

In [ ]:
# simpleapi.py
import os
from flask import Flask
from flask_restful import Resource, Api
from secure_check import authenticate, identity
from flask_jwt import JWT, jwt_required
from flask_sqlalchemy import SQLAlchemy
from flask_migrate import Migrate 

app = Flask(__name__)
app.config['SECRET_KEY'] = 'secretkey'
basedir = os.path.abspath(os.path.dirname(__file__))
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///'+os.path.join(basedir, 'datasqlite')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)
Migrate(app,db)

api = Api(app)
jwt = JWT(app, authenticate, identity)

class Puppy(db.Model):
    
    name = db.Column(db.String(80), primary_key = True)
    
    def __init__(self,name):
        self.name = name
        
    def json(self):
        return {'name':self.name}
    
class PuppyNames(Resource):
    def get(self, name):
        pup = Puppy.query.filter_by(name = name).first()
        
        if pup:
            return pup.json()
        else:
            return {'name': None}
    
    def post(self,name):
        
        pup = Puppy(name = name)
        db.session.add(pup)
        db.session.commit()
        
        return pup.json()
    
    def delete(self, name):
        pup = Puppy.query.filter_by(name = name).first()
        db.session.delete(pup)
        db.session.commit()
        
        return {'note': 'delete success'}
    
class AllNames(Resource):
    def get(self):
        puppies = Puppy.query.all()
        
        return [pup.json() for pup in puppies]
    

Then you need to perform a migration.

In [ ]:
export FLASK_APP=simpleapi.py
 
flask db init

flask db migrate -m "first migrate"

flask db upgrade

## Deploying Your Application!!

![SegmentLocal](https://media3.giphy.com/media/5GoVLqeAOo6PK/giphy.gif "segment")

Everything we have done so far has made local flask apps, but you want a live web app that can be accessed by everyone!!!

We are going to do this using __Heroku__

You have your application, for simplicity sake, here is my example:

In [ ]:
# app.py
from flask import Flask 

app = Flask(__name__)

@app.route('/')
def index():
    return "Hello World!"

if __name__ == '__main__':
    app.run()

Now you need to create your virtual environment at the command line:

In [ ]:
conda create -n flaskdeploy flask 
# Name it whatever you please
source activate flaskdeploy

This is where you should install any libraries you will need for your application, such as flask_login or flask_migrate 

Now on command line:

In [ ]:
python app.py

You can see your app running in the browser now and check for errors,

so back on the command line:

In [ ]:
pip install gunicorn
# Allows us to correctly render this on the web
# Does a lot of behind the scenes work

In [ ]:
pip freeze > requirements.txt

You will now have a requirements.txt file with everything you need!

Create a new file called Procfile, that contains:

In [ ]:
web: gunicorn app:app

Now go to google and search Heroku CLI

It will require you to install git if you have not already, then download heroku

Now go to www.heroku.com, and create an account

It will take you to your dashboard, where you can hit "Create New App" and then name it

This will instantly create the app, now we need to return to your command line and type:

In [ ]:
heroku login

After this it will give you a spot to enter your email and password

Then make sure you cd into your project folder if thats not where you are right now

In [ ]:
git init 

heroku git:remote -a appname
# Appname is the name of YOUR app

git add .

git commit -am "first commit"

git push heroku master

Finishing these steps will deploy your application!!!! 

It will give you a URL and clicking on this will allow you to see it live on the web!!!